# Code

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

### Instantiation of all the constants : 

In [3]:
#Constants
Ts = 0.1
thymio_speed_to_mms = 0.341
distance_wheel = 95

#Thymio goes forward
q_F_x = 0.008
q_F_y =  0.008
q_F_theta = 0.000006
q_F_x_dot = 0.779  
q_F_y_dot =  0.779  
q_F_theta_dot = 0.000591

Q_F = np.array([[q_F_x, 0, 0, 0, 0, 0], 
                [0, q_F_y, 0, 0, 0, 0], 
                [0, 0, q_F_theta, 0, 0, 0],
                [0, 0, 0, q_F_x_dot, 0, 0], 
                [0, 0, 0, 0, q_F_y_dot, 0], 
                [0, 0, 0, 0, 0, q_F_theta_dot]]);

r_F_x_dot = 0.779  
r_F_y_dot = 0.779
r_F_theta_dot = 0.000591

R_F = np.array([[r_F_x_dot, 0, 0],
               [0, r_F_y_dot, 0],
               [0, 0, r_F_theta_dot]]);

A_F = np.array([[1, 0, 0, Ts, 0, 0], 
                [0, 1, 0, 0, Ts, 0], 
                [0, 0, 1, 0, 0, 0],
                [0, 0, 0, 1, 0, 0], 
                [0, 0, 0, 0, 1, 0], 
                [0, 0, 0, 0, 0, 1]]);


#Thymio rotates
q_R_x = 0.01
q_R_y =  0.01
q_R_theta = 0.00001 
q_R_x_dot = 1.31
q_R_y_dot =  1.31
q_R_theta_dot = 0.00122

Q_R = np.array([[q_R_x, 0, 0, 0, 0, 0], 
                [0, q_R_y, 0, 0, 0, 0], 
                [0, 0, q_R_theta, 0, 0, 0],
                [0, 0, 0, q_R_x_dot, 0, 0], 
                [0, 0, 0, 0, q_R_y_dot, 0], 
                [0, 0, 0, 0, 0, q_R_theta_dot]]);

r_R_x_dot = 1.31
r_R_y_dot = 1.31
r_R_theta_dot = 0.00122  

R_R = np.array([[r_R_x_dot, 0, 0],
               [0, r_R_y_dot, 0],
               [0, 0, r_R_theta_dot]]);

A_R = np.array([[1, 0, 0, 0, 0, 0], 
                [0, 1, 0, 0, 0, 0], 
                [0, 0, 1, 0, 0, Ts],
                [0, 0, 0, 1, 0, 0], 
                [0, 0, 0, 0, 1, 0], 
                [0, 0, 0, 0, 0, 1]]);

# commun matrix
H = np.array([[0, 0, 0, 1, 0, 0], 
              [0, 0, 0, 0, 1, 0],
              [0, 0, 0, 0, 0, 1]]);



### Code of Kalman function :

In [4]:
def kalman_filter(meas_speed_left, meas_speed_right, x_est_prev, P_est_prev, Q, R, A):
    
    
    ## Prediciton through the a priori estimate
    # estimated mean of the state
    x_est_a_priori = np.dot(A, x_est_prev) 
    
    # Estimated covariance of the state
    P_est_a_priori = np.dot(A, np.dot(P_est_prev, A.T))
    P_est_a_priori = P_est_a_priori + Q
    
    
    #measurements
    speed_trans = (meas_speed_left + meas_speed_right) * thymio_speed_to_mms / 2
    speed_rot =  (meas_speed_right - meas_speed_left) * (thymio_speed_to_mms / distance_wheel)
    
    theta_est = x_est_a_priori[2];
    x_dot = speed_trans * np.cos(theta_est)
    y_dot = speed_trans * np.sin(theta_est)
    theta_dot = speed_rot
    
    y = np.array([[x_dot], [y_dot], [theta_dot]])
   

    # innovation / measurement residual
    i = y - np.dot(H, x_est_a_priori)
    # measurement prediction covariance
    S = np.dot(H, np.dot(P_est_a_priori, H.T)) + R
             
    # Kalman gain (tells how much the predictions should be corrected based on the measurements)
    K = np.dot(P_est_a_priori, np.dot(H.T, np.linalg.inv(S)))
    
    
    # a posteriori estimate
    x_est = x_est_a_priori + np.dot(K,i)
    P_est = P_est_a_priori - np.dot(K,np.dot(H, P_est_a_priori))
     
    return x_est, P_est

### Utilization of Kalman : 

In [ ]:
# Import tdmclient Notebook environment:
import tdmclient.notebook
await tdmclient.notebook.start()

In [ ]:
camera_on = False

In [ ]:
#initial state :
x0 = 0
y0 = 0
x_dot0 = 0 
y_dot0 = 0
theta_dot0 = 0 

x_est = [np.array([[x0], [y0], [theta0], [x_dot0], [y_dot0], [theta_dot0]])]
P_est = [0.001 * np.ones(6)]

step = 0
otpimal_path = [[100, 100], [100, 100], [100, 100], [100, 100], [100, 100]]

In [ ]:
from threading import Timer

class RepeatedTimer(object):
    def __init__(self, interval, function, *args):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False

In [ ]:
@tdmclient.notebook.sync_var
def get_speed():
    return motor_left_speed, motor_right_speed

def kalman_call():
    global counter
    
    #retrieves the equivalent of the command that was applied 100ms ago
    speed_left = optimal_command[counter][0]
    speed_right = optimal_command[counter][1]
    
    speed_trans = (speed_left + speed_right) * thymio_speed_to_mms / 2
    speed_rot =  (speed_right - speed_left) * (thymio_speed_to_mms / distance_wheel)
    theta = x_est[-1][2];
    
    x_est[3] = speed_trans * np.cos(theta)
    x_est[4] = speed_trans * np.sin(theta)
    x_est[5] = speed_rot
    
    #gets the right matrix corresponding to the command :
    #Thymio goes forward
    if speed_left == speed_right :     
        Q = Q_F
        R = R_F
        A = A_F
    
    #Thymio is rotating
    else :
        Q = Q_R
        R = R_R
        A = A_R
    
    #measures the speed of the robot
    meas_speed_left,  meas_speed_right = get_speed()
    
    #computes the estimated states
    new_x_est, new_P_est = kalman_filter(meas_speed_left, meas_speed_right, x_est_prev, P_est_prev, Q, R, A)
    
    x_est.append(new_x_est)
    P_est.append(new_P_est)
    
    counter += 1
    
    

In [ ]:
%%run_python

@onevent
def move(speed_left, speed_right):
    global motor_left_target, motor_right_target
    motor_left_target = speed_left
    motor_right_target = speed_right

In [ ]:
def send_command()
    send_event("move", optimal_command[step + 1][0], optimal_command[step + 1][1])

In [ ]:
#start of Kalman
while camera_on != True :
    if step == 0:
        send_event("move", optimal_command[0][0], optimal_command[0][1])
         
    timer_kalman = RepeatedTimer(Ts, kalman_call)
    timer_command = RepeatedTimer(Ts, send_command)
    
timer_kalman.stop()
timer_command.stop()